# Running uniqed on a chirp signal

In [46]:
# Animation class definition

from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

from uniqed.runners.tof_run import detect_outlier

class TOFAnimation:
    def __init__(self, data, n_range=range(1,25), min_tof=1.5):
        self.data = data
        self.n_range = n_range
        self.min_tof = min_tof
        self.fig, self.ax = plt.subplots(figsize=(8,8))
        self.anim = animation.FuncAnimation(
            self.fig, self.animate,
            init_func=self.setup_plot,
            frames=self.n_range,
            interval=500,
            blit=True
        )

    def setup_plot(self):
        self.plot, = self.ax.plot(self.data, 'c-')
        self.outliers, = self.ax.plot([], 'ro')
        self.annotations = []
        return [self.plot, self.outliers]

    def animate(self, n):
        res_df = detect_outlier(pd.DataFrame(self.data), cutoff_n=n)
        outliers = res_df.query(f"TOF>{self.min_tof}")[0]
        print(outliers.shape)
        outlier_tof = res_df.query(f"TOF>{self.min_tof}")['TOF_score']
        self.outliers.set_ydata(outliers)
        # Clear old annotations
        for ann in self.annotations:
            ann.remove()
        self.annotations = []
        # Generate new annotations
        self.annotations.extend([
            self.ax.text(
                0.01,0.06,
                f'n_neighbors = {n}',
                transform=self.ax.transAxes),
            self.ax.text(
                0.01,0.02,
                f'(min TOF to consider outlier = {self.min_tof})',
                transform=self.ax.transAxes)
        ])
        for i, lof in enumerate(outlier_lof):
            try:
                self.annotations.append(
                    self.ax.annotate(round(tof,2), outliers[i,:])
                )
            except:
                print(lof, outliers[i,:])
        return [self.plot, self.outliers]
    
    def save(self, *args, **kwargs):
        return self.anim.save(*args, **kwargs)
    
    def show(self, *args, **kwargs):
        return HTML(self.anim.to_html5_video(*args, **kwargs))

In [47]:
# Read data

import numpy as np
import pandas as pd

from gwpy.timeseries import TimeSeries

fn = 'data/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5'
strain = TimeSeries.read(fn,format='hdf5.losc')
data = np.array(strain.data)[::1000] # downsample
scatter_data = np.stack((np.arange(0,data.size), data), axis=-1)

anim = TOFAnimation(data, n_range=range(10,50), min_tof=1)
plt.close()
anim.show()

                  0     TOF_score  TOF
0      1.644585e-19           NaN  NaN
1      2.389694e-19  12297.558010  0.0
2     -3.135705e-21  11013.435420  0.0
3     -1.291181e-20   8417.043587  0.0
4      6.726399e-21  10915.238053  0.0
...             ...           ...  ...
16773 -2.418374e-19  10380.318793  0.0
16774  2.534213e-19   5762.980501  0.0
16775 -1.880368e-19   5842.225197  0.0
16776  4.935541e-19  10352.385341  0.0
16777 -4.603878e-19           NaN  NaN

[16778 rows x 3 columns]
(0,)
                  0     TOF_score  TOF
0      1.644585e-19           NaN  NaN
1      2.389694e-19  12297.558010  0.0
2     -3.135705e-21  11013.435420  0.0
3     -1.291181e-20   8417.043587  0.0
4      6.726399e-21  10915.238053  0.0
...             ...           ...  ...
16773 -2.418374e-19  10380.318793  0.0
16774  2.534213e-19   5762.980501  0.0
16775 -1.880368e-19   5842.225197  0.0
16776  4.935541e-19  10352.385341  0.0
16777 -4.603878e-19           NaN  NaN

[16778 rows x 3 columns]
(0,)
  

                  0     TOF_score  TOF
0      1.644585e-19           NaN  NaN
1      2.389694e-19  12297.558010  0.0
2     -3.135705e-21  11013.435420  0.0
3     -1.291181e-20   8417.043587  0.0
4      6.726399e-21  10915.238053  0.0
...             ...           ...  ...
16773 -2.418374e-19  10380.318793  0.0
16774  2.534213e-19   5762.980501  0.0
16775 -1.880368e-19   5842.225197  0.0
16776  4.935541e-19  10352.385341  0.0
16777 -4.603878e-19           NaN  NaN

[16778 rows x 3 columns]
(0,)
                  0     TOF_score  TOF
0      1.644585e-19           NaN  NaN
1      2.389694e-19  12297.558010  0.0
2     -3.135705e-21  11013.435420  0.0
3     -1.291181e-20   8417.043587  0.0
4      6.726399e-21  10915.238053  0.0
...             ...           ...  ...
16773 -2.418374e-19  10380.318793  0.0
16774  2.534213e-19   5762.980501  0.0
16775 -1.880368e-19   5842.225197  0.0
16776  4.935541e-19  10352.385341  0.0
16777 -4.603878e-19           NaN  NaN

[16778 rows x 3 columns]
(0,)
  

                  0     TOF_score  TOF
0      1.644585e-19           NaN  NaN
1      2.389694e-19  12297.558010  0.0
2     -3.135705e-21  11013.435420  0.0
3     -1.291181e-20   8417.043587  0.0
4      6.726399e-21  10915.238053  0.0
...             ...           ...  ...
16773 -2.418374e-19  10380.318793  0.0
16774  2.534213e-19   5762.980501  0.0
16775 -1.880368e-19   5842.225197  0.0
16776  4.935541e-19  10352.385341  0.0
16777 -4.603878e-19           NaN  NaN

[16778 rows x 3 columns]
(0,)
                  0     TOF_score  TOF
0      1.644585e-19           NaN  NaN
1      2.389694e-19  12297.558010  0.0
2     -3.135705e-21  11013.435420  0.0
3     -1.291181e-20   8417.043587  0.0
4      6.726399e-21  10915.238053  0.0
...             ...           ...  ...
16773 -2.418374e-19  10380.318793  0.0
16774  2.534213e-19   5762.980501  0.0
16775 -1.880368e-19   5842.225197  0.0
16776  4.935541e-19  10352.385341  0.0
16777 -4.603878e-19           NaN  NaN

[16778 rows x 3 columns]
(0,)
  